IMPORTING DEPENDENCIES

In [121]:
!pip install yahoo_fin


In [122]:
!pip install ta

In [182]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
import torch.nn.utils.rnn as rnn
import torch.nn.functional as F
import torch.nn as nn
import torch
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from sklearn.linear_model import LinearRegression
from tensorflow.keras.layers import Input, Dense, Masking
from sklearn.preprocessing import MinMaxScaler
from yahoo_fin import news
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GRU, Dense, Dropout, LSTM
from tensorflow.keras.losses import MeanSquaredError



TESLA AND APPLE STOCK PRICE NEWS DATA

In [124]:
data = pd.read_csv('/content/combined.csv')
index_values0 = data.loc[data['ticker'] == 'AAPL']
index_values1 = data.loc[data['ticker'] == 'TSLA']

In [125]:
index_values0.head()

,ticker,headline,date,eventRet,Close
22,AAPL,Despite a wave of analyst love regarding yeste...,2011-06-07,-0.032871,10.093049
23,AAPL,Bill Ackman's hedge fund made ~$490M in one da...,2011-06-14,0.002913,10.105204
28,AAPL,Apple (AAPL -1.5%) shares continue their perpl...,2011-06-20,0.001156,9.584808
47,AAPL,"After lagging for months, tech bellwethers App...",2011-07-07,0.029160,10.857841
57,AAPL,"Looking at a ""refresh of virtually every produ...",2011-07-13,0.010666,10.882765


In [126]:
index_values1.head()

,ticker,headline,date,eventRet,Close
18,TSLA,Tesla Motors (TSLA +3.6%) announces a follow-o...,2011-05-25,0.098105,1.932000
21,TSLA,Tesla Motors (TSLA +6.3%) jumps premarket afte...,2011-06-03,0.008590,2.008667
71,TSLA,Tesla Motors (TSLA +4.5%) scores a much bigger...,2011-07-20,0.053271,1.912667
103,TSLA,A video rumored to be released back-channels b...,2011-08-12,0.099013,1.754000
141,TSLA,Tesla Motors (TSLA +1.5%) gains on positive co...,2011-09-21,-0.005349,1.723333


In [127]:
news0 = index_values0[['headline','date','eventRet']]
news1 = index_values1[['headline','date','eventRet']]

In [128]:
news0['Date'] = news0['date']
news1['Date'] = news1['date']

<ipython-input-128-4546fdcd0dd7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news0['Date'] = news0['date']
<ipython-input-128-4546fdcd0dd7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news1['Date'] = news1['date']


In [129]:
news0 = news0.drop('date', axis = 1)
news1 = news1.drop('date', axis = 1)

In [130]:
news0.head()

,headline,eventRet,Date
22,Despite a wave of analyst love regarding yeste...,-0.032871,2011-06-07
23,Bill Ackman's hedge fund made ~$490M in one da...,0.002913,2011-06-14
28,Apple (AAPL -1.5%) shares continue their perpl...,0.001156,2011-06-20
47,"After lagging for months, tech bellwethers App...",0.029160,2011-07-07
57,"Looking at a ""refresh of virtually every produ...",0.010666,2011-07-13


In [131]:
news1.head()

,headline,eventRet,Date
18,Tesla Motors (TSLA +3.6%) announces a follow-o...,0.098105,2011-05-25
21,Tesla Motors (TSLA +6.3%) jumps premarket afte...,0.008590,2011-06-03
71,Tesla Motors (TSLA +4.5%) scores a much bigger...,0.053271,2011-07-20
103,A video rumored to be released back-channels b...,0.099013,2011-08-12
141,Tesla Motors (TSLA +1.5%) gains on positive co...,-0.005349,2011-09-21


TESLA AND APPLE STOCK PRICE DATA

In [132]:
# Define top companies (e.g., Apple, Microsoft, Amazon, Google, Nvidia)
top_companies0 = [ 'AAPL']
top_companies1 = ['TSLA']
# Download historical stock data
stock_data0 = yf.download(top_companies0, start="2011-05-02", end="2018-12-24", interval="1d")
stock_data1 = yf.download(top_companies1, start="2011-05-02", end="2018-12-24", interval="1d")
# Save as CSV
stock_data0.to_csv("top_sp500_stock_data.csv")
stock_data1.to_csv("top_sp500_stock_data.csv")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [133]:
stock_data0 = stock_data0.droplevel(1, axis=1)
stock_data1 = stock_data1.droplevel(1, axis=1)

In [134]:
stock_data0 = stock_data0.reset_index()
stock_data1 = stock_data1.reset_index()

In [135]:
stock_data0.head()

Price,Date,Close,High,Low,Open,Volume
0,2011-05-02,10.420812,10.546904,10.397339,10.524935,442713600
1,2011-05-03,10.478591,10.529449,10.400949,10.472272,313348000
2,2011-05-04,10.519820,10.587832,10.438868,10.480397,389250400
3,2011-05-05,10.434957,10.561349,10.413891,10.484610,335969200
4,2011-05-06,10.432246,10.532759,10.418704,10.523430,280134400


In [136]:
stock_data1.head()

Price,Date,Close,High,Low,Open,Volume
0,2011-05-02,1.830000,1.853333,1.804000,1.840000,11769000
1,2011-05-03,1.791333,1.826000,1.766667,1.825333,13708500
2,2011-05-04,1.779333,1.800000,1.716667,1.785333,15667500
3,2011-05-05,1.762667,1.829333,1.744667,1.813333,18277500
4,2011-05-06,1.808000,1.846667,1.774667,1.793333,14725500


CONCATENATING NEWS AND STOCK PRICE DATA ACCORDING TO DATE

In [137]:
import pandas as pd

# Ensure both DataFrames have "Date" in datetime format
stock_data0["Date"] = pd.to_datetime(stock_data0["Date"], errors="coerce")
news0["Date"] = pd.to_datetime(news0["Date"], errors="coerce")  # Fix column name if needed

# Merge the DataFrames on 'Date', keeping only common dates
merged_df0 = pd.merge(stock_data0, news0, on="Date", how="inner")

# Sort by Date and reset index
merged_df0 = merged_df0.sort_values(by="Date").reset_index(drop=True)

merged_df0.head()

,Date,Close,High,Low,Open,Volume,headline,eventRet
0,2011-06-07,9.992278,10.178257,9.988065,10.176752,529785200,Despite a wave of analyst love regarding yeste...,-0.032871
1,2011-06-14,10.004314,10.028690,9.910121,9.930886,334569200,Bill Ackman's hedge fund made ~$490M in one da...,0.002913
2,2011-06-20,9.489112,9.560735,9.344061,9.550503,640645600,Apple (AAPL -1.5%) shares continue their perpl...,0.001156
3,2011-07-07,10.749433,10.773507,10.653132,10.673296,399663600,"After lagging for months, tech bellwethers App...",0.029160
4,2011-07-13,10.774110,10.833695,10.724755,10.783438,391638800,"Looking at a ""refresh of virtually every produ...",0.010666


In [138]:
import pandas as pd

# Ensure both DataFrames have "Date" in datetime format
stock_data1["Date"] = pd.to_datetime(stock_data1["Date"], errors="coerce")
news1["Date"] = pd.to_datetime(news1["Date"], errors="coerce")  # Fix column name if needed

# Merge the DataFrames on 'Date', keeping only common dates
merged_df1 = pd.merge(stock_data1, news1, on="Date", how="inner")

# Sort by Date and reset index
merged_df1 = merged_df1.sort_values(by="Date").reset_index(drop=True)

merged_df1.head()

,Date,Close,High,Low,Open,Volume,headline,eventRet
0,2011-05-25,1.932000,1.934000,1.744667,1.793333,70396500,Tesla Motors (TSLA +3.6%) announces a follow-o...,0.098105
1,2011-06-03,2.008667,2.100000,1.966667,1.996667,93138000,Tesla Motors (TSLA +6.3%) jumps premarket afte...,0.008590
2,2011-07-20,1.912667,2.029333,1.853333,1.866667,45724500,Tesla Motors (TSLA +4.5%) scores a much bigger...,0.053271
3,2011-08-12,1.754000,1.809333,1.690667,1.706667,15136500,A video rumored to be released back-channels b...,0.099013
4,2011-09-21,1.723333,1.796667,1.713333,1.730000,14814000,Tesla Motors (TSLA +1.5%) gains on positive co...,-0.005349


In [139]:
merged_df0.dropna(inplace = True)
merged_df1.dropna(inplace = True)

In [140]:
merged_df0.shape

(462, 8)

In [141]:
merged_df1.shape

(479, 8)

ADDING NEW FEATURE

In [142]:
merged_df0['Daily_Return'] = stock_data0['Close'].pct_change()
merged_df1['Daily_Return'] = stock_data1['Close'].pct_change()

In [143]:
## EXPONENTIAL MOVING AVERAGE
merged_df0['EMA_100'] = merged_df0['Close'].ewm(span=10, adjust=False).mean()
merged_df1['EMA_100'] = merged_df1['Close'].ewm(span=10, adjust=False).mean()
## BOLLINGER DEMANDS
import ta
merged_df0['RSI'] = ta.momentum.RSIIndicator(merged_df0['Close'], window=14).rsi()
merged_df1['RSI'] = ta.momentum.RSIIndicator(merged_df1['Close'], window=14).rsi()
## Volatility
merged_df0['Volatility_10'] = merged_df0['Daily_Return'].rolling(window=10).std()
merged_df1['Volatility_10'] = merged_df1['Daily_Return'].rolling(window=10).std()

In [144]:
price0 = merged_df0.dropna()
price1 = merged_df1.dropna()

In [145]:
price0.head()

,Date,Close,High,Low,Open,Volume,headline,eventRet,Daily_Return,EMA_100,RSI,Volatility_10
14,2011-08-25,11.246580,11.298642,10.984164,10.986571,871346000,"Let's get real: In the long run, Apple (AAPL -...",0.026750,-0.015593,11.172053,63.015423,0.011054
15,2011-08-25,11.246580,11.298642,10.984164,10.986571,871346000,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,0.026750,-0.002446,11.185604,63.015423,0.010997
16,2011-08-26,11.543303,11.549923,11.158707,11.169841,641477200,Apple (AAPL +2.6%) shares rallied today to ful...,0.036503,-0.006609,11.250640,68.061735,0.010750
17,2011-09-19,12.387427,12.435577,11.892991,11.947158,823860800,Apple (AAPL +2%) is outperforming strongly tod...,0.051400,0.013817,11.457329,77.476789,0.012255
18,2011-09-26,12.132835,12.157211,11.775625,12.033225,812876400,"U.S. shares are higher across the board, with ...",-0.006321,-0.005285,11.580148,70.706688,0.012269


In [146]:
price1.head()

,Date,Close,High,Low,Open,Volume,headline,eventRet,Daily_Return,EMA_100,RSI,Volatility_10
13,2012-03-22,2.293333,2.343333,2.286667,2.331333,7836000,Luxury electric auto maker Tesla Motors (TSLA ...,-0.025205,0.070209,1.935856,62.384754,0.034413
14,2012-03-26,2.493333,2.539333,2.336000,2.372667,47107500,Tesla Motors (TSLA +3.7%) is upgraded to Buy f...,0.102554,-0.008156,2.037215,66.732863,0.034030
15,2012-04-03,2.534000,2.564667,2.444667,2.446667,16471500,Shares of Tesla Motors (TSLA +2.9%) kick up a ...,-0.057820,-0.041116,2.127540,67.554142,0.035399
16,2012-04-09,2.210000,2.286000,2.206667,2.273333,24835500,Abu Dhabi National Energy sells off its stake ...,-0.074245,-0.003729,2.142532,55.746075,0.034803
17,2012-05-09,2.004000,2.051333,1.984000,2.020000,29218500,More on Tesla Motors (NASDAQ:TSLA): Q1 misses ...,0.021949,0.084581,2.117345,49.787353,0.041871


In [147]:
price0['Volume'] = price0['Volume']/10e6
price1['Volume'] = price1['Volume']/10e6

<ipython-input-147-83dc1eb114d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price0['Volume'] = price0['Volume']/10e6
<ipython-input-147-83dc1eb114d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price1['Volume'] = price1['Volume']/10e6


Performing Roubust Scaling on data

In [148]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cs0 = scaler.fit_transform( price0[['Close','High','Low','Open','Volume','eventRet',
                                                                              'Daily_Return','EMA_100','RSI','Volatility_10']])
df0 = pd.DataFrame(cs0)

In [149]:
cs1 = scaler.fit_transform( price1[['Close','High','Low','Open','Volume','eventRet',
                                                                              'Daily_Return','EMA_100','RSI','Volatility_10']])
df1 = pd.DataFrame(cs1)

Assigning Features Name

In [150]:
df0.rename(columns={0:'Close',1:'High',2:'Low',3:'Open',
                   4: 'Volume',5:'eventRet',6:'Daily_Return',
                   7:'EMA_100', 8:'RSI', 9: 'Volatility_10'}, inplace=True)

In [151]:
df1.rename(columns={0:'Close',1:'High',2:'Low',3:'Open',
                   4: 'Volume',5:'eventRet',6:'Daily_Return',
                   7:'EMA_100', 8:'RSI', 9: 'Volatility_10'}, inplace=True)

In [152]:
price0 = price0.reset_index()
price1 = price1.reset_index()

In [153]:
df0['headline']	= price0['headline']
df0['date'] = price0['Date']

In [154]:
df1['headline']	= price1['headline']
df1['date'] = price1['Date']

In [155]:
df0.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,-0.632381,-0.638991,-0.635903,-0.640852,1.474185,0.498016,-0.732289,-0.619691,0.163419,-0.496497,"Let's get real: In the long run, Apple (AAPL -...",2011-08-25
1,-0.632381,-0.638991,-0.635903,-0.640852,1.474185,0.498016,-0.115541,-0.619120,0.163419,-0.502681,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,2011-08-25
2,-0.619890,-0.628485,-0.628543,-0.633196,0.941926,0.682898,-0.310802,-0.616377,0.366426,-0.529206,Apple (AAPL +2.6%) shares rallied today to ful...,2011-08-26
3,-0.584355,-0.591453,-0.597582,-0.600724,1.364234,0.965288,0.647375,-0.607660,0.745181,-0.367130,Apple (AAPL +2%) is outperforming strongly tod...,2011-09-19
4,-0.595072,-0.603092,-0.602530,-0.597128,1.338799,-0.128882,-0.248733,-0.602480,0.472829,-0.365674,"U.S. shares are higher across the board, with ...",2011-09-26


In [156]:
df1.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,-1.575085,-1.639529,-1.560488,-1.582101,-1.005321,-0.232765,2.002755,-1.853741,0.384418,0.362720,Luxury electric auto maker Tesla Motors (TSLA ...,2012-03-22
1,-1.549470,-1.613812,-1.554022,-1.576701,-0.620771,1.140055,-0.232651,-1.838819,0.595292,0.340985,Tesla Motors (TSLA +3.7%) is upgraded to Buy f...,2012-03-26
2,-1.544261,-1.610488,-1.539779,-1.567033,-0.920761,-0.583233,-1.172859,-1.825523,0.635122,0.418674,Shares of Tesla Motors (TSLA +2.9%) kick up a ...,2012-04-03
3,-1.585758,-1.647051,-1.570973,-1.589679,-0.838860,-0.759717,-0.106366,-1.823316,0.062457,0.384876,Abu Dhabi National Energy sells off its stake ...,2012-04-09
4,-1.612142,-1.677841,-1.600157,-1.622776,-0.795942,0.273919,2.412745,-1.827024,-0.226528,0.786095,More on Tesla Motors (NASDAQ:TSLA): Q1 misses ...,2012-05-09


Using Finbert to performing sentiment analysis on stock price news data

In [157]:
!pip install feedparser

In [158]:
import feedparser
import pandas as pd

In [159]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load FinBERT model (trained for financial sentiment analysis)
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


In [160]:
df0.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,-0.632381,-0.638991,-0.635903,-0.640852,1.474185,0.498016,-0.732289,-0.619691,0.163419,-0.496497,"Let's get real: In the long run, Apple (AAPL -...",2011-08-25
1,-0.632381,-0.638991,-0.635903,-0.640852,1.474185,0.498016,-0.115541,-0.619120,0.163419,-0.502681,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,2011-08-25
2,-0.619890,-0.628485,-0.628543,-0.633196,0.941926,0.682898,-0.310802,-0.616377,0.366426,-0.529206,Apple (AAPL +2.6%) shares rallied today to ful...,2011-08-26
3,-0.584355,-0.591453,-0.597582,-0.600724,1.364234,0.965288,0.647375,-0.607660,0.745181,-0.367130,Apple (AAPL +2%) is outperforming strongly tod...,2011-09-19
4,-0.595072,-0.603092,-0.602530,-0.597128,1.338799,-0.128882,-0.248733,-0.602480,0.472829,-0.365674,"U.S. shares are higher across the board, with ...",2011-09-26


In [161]:
df1.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,-1.575085,-1.639529,-1.560488,-1.582101,-1.005321,-0.232765,2.002755,-1.853741,0.384418,0.362720,Luxury electric auto maker Tesla Motors (TSLA ...,2012-03-22
1,-1.549470,-1.613812,-1.554022,-1.576701,-0.620771,1.140055,-0.232651,-1.838819,0.595292,0.340985,Tesla Motors (TSLA +3.7%) is upgraded to Buy f...,2012-03-26
2,-1.544261,-1.610488,-1.539779,-1.567033,-0.920761,-0.583233,-1.172859,-1.825523,0.635122,0.418674,Shares of Tesla Motors (TSLA +2.9%) kick up a ...,2012-04-03
3,-1.585758,-1.647051,-1.570973,-1.589679,-0.838860,-0.759717,-0.106366,-1.823316,0.062457,0.384876,Abu Dhabi National Energy sells off its stake ...,2012-04-09
4,-1.612142,-1.677841,-1.600157,-1.622776,-0.795942,0.273919,2.412745,-1.827024,-0.226528,0.786095,More on Tesla Motors (NASDAQ:TSLA): Q1 misses ...,2012-05-09


In [162]:
def get_sentiment(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get label with highest probability
    probs = F.softmax(outputs.logits, dim=-1)

    # Convert label to sentiment
    return (probs[0][2]-probs[0][0]).numpy().item() ## positive - negative

In [163]:
df0['sentiment'] = df0['headline'].apply(get_sentiment)
df1['sentiment'] = df1['headline'].apply(get_sentiment)

In [164]:
df0 = df0.drop(['headline','date'], axis=1)
df1 = df1.drop(['headline','date'], axis=1)

In [165]:
df0.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,sentiment
0,-0.632381,-0.638991,-0.635903,-0.640852,1.474185,0.498016,-0.732289,-0.619691,0.163419,-0.496497,0.415604
1,-0.632381,-0.638991,-0.635903,-0.640852,1.474185,0.498016,-0.115541,-0.619120,0.163419,-0.502681,0.008747
2,-0.619890,-0.628485,-0.628543,-0.633196,0.941926,0.682898,-0.310802,-0.616377,0.366426,-0.529206,-0.926992
3,-0.584355,-0.591453,-0.597582,-0.600724,1.364234,0.965288,0.647375,-0.607660,0.745181,-0.367130,-0.538386
4,-0.595072,-0.603092,-0.602530,-0.597128,1.338799,-0.128882,-0.248733,-0.602480,0.472829,-0.365674,-0.920180


In [166]:
df1.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,sentiment
0,-1.575085,-1.639529,-1.560488,-1.582101,-1.005321,-0.232765,2.002755,-1.853741,0.384418,0.362720,0.015583
1,-1.549470,-1.613812,-1.554022,-1.576701,-0.620771,1.140055,-0.232651,-1.838819,0.595292,0.340985,-0.842692
2,-1.544261,-1.610488,-1.539779,-1.567033,-0.920761,-0.583233,-1.172859,-1.825523,0.635122,0.418674,-0.918571
3,-1.585758,-1.647051,-1.570973,-1.589679,-0.838860,-0.759717,-0.106366,-1.823316,0.062457,0.384876,0.536333
4,-1.612142,-1.677841,-1.600157,-1.622776,-0.795942,0.273919,2.412745,-1.827024,-0.226528,0.786095,-0.915175


In [167]:
df0.shape

(449, 11)

In [168]:
df1.shape

(466, 11)

In [169]:
X_train0 = df0[:350]
X_valid0 = df0[350:]

Creating time series sequence as input to GRU and LSTM

In [170]:
def create_seq(df, seq_len):
  x_train = []
  y_train = []
  for i in range(len(df)-seq_len):
    x_train.append(df.iloc[i:(i+seq_len)])
    y_train.append(df.iloc[i+seq_len,0])
  return np.array(x_train), np.array(y_train)

In [171]:
x_train0, y_train0 = create_seq(X_train0, seq_len = 60)
x_valid0, y_valid0 = create_seq(X_valid0, seq_len = 60)

In [172]:
x1, y1 = create_seq(df1, seq_len = 60)

In [173]:
!pip install transformers

GRU MODEL

In [174]:
model= tf.keras.Sequential([
    GRU(32, return_sequences=True, kernel_regularizer=l2(0.001)),  # L2 Regularization
    Dropout(0.3),  # 30% neurons ko random drop karenge
    GRU(16, return_sequences=False, kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(1)  # Final output layer
])
model.compile(loss='mean_squared_error',optimizer='adam')

In [175]:
model.fit(x_train0, y_train0, validation_data = (x_valid0, y_valid0), epochs = 25)

Epoch 1/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.2010 - val_loss: 0.2788
Epoch 2/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1282 - val_loss: 0.1216
Epoch 3/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0904 - val_loss: 0.1062
Epoch 4/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0945 - val_loss: 0.1035
Epoch 5/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0787 - val_loss: 0.1007
Epoch 6/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0699 - val_loss: 0.0895
Epoch 7/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0742 - val_loss: 0.1063
Epoch 8/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0664 - val_loss: 0.0977
Epoch 9/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0600 - val_loss: 0.0923
Epoch 10/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0542 - val_loss: 0.0724
Epoch 11/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0533 - val_loss: 0.0718
Epoch 12/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0

In [176]:
y_pred1 = model.predict(x1)
y_pred0 = model.predict(x_valid0)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [177]:
mse = tf.keras.losses.MeanSquaredError()

In [178]:
mean_absolute_error0 = mse(y_pred0, y_valid0)
mean_absolute_error1 = mse(y_pred1, y1)

In [179]:
mean_absolute_error0

<tf.Tensor: shape=(), dtype=float32, numpy=0.024909665808081627>

In [180]:
mean_absolute_error1

<tf.Tensor: shape=(), dtype=float32, numpy=0.02947545237839222>

LSTM MODEL

In [183]:
model1= tf.keras.Sequential([
    LSTM(32, return_sequences=True, kernel_regularizer=l2(0.001)),  # L2 Regularization
    Dropout(0.3),  # 30% neurons ko random drop karenge
    LSTM(16, return_sequences=False, kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(1)  # Final output layer
])
model1.compile(loss='mean_squared_error',optimizer='adam')

In [185]:
model1.fit(x_train0, y_train0, validation_data = (x_valid0, y_valid0), epochs = 25)

Epoch 1/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - loss: 0.2800 - val_loss: 0.2519
Epoch 2/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.1076 - val_loss: 0.0963
Epoch 3/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0909 - val_loss: 0.0939
Epoch 4/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0739 - val_loss: 0.1136
Epoch 5/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0729 - val_loss: 0.1016
Epoch 6/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0677 - val_loss: 0.0876
Epoch 7/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0651 - val_loss: 0.0953
Epoch 8/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0596 - val_loss: 0.0876
Epoch 9/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0550 - val_loss: 0.0766
Epoch 10/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0537 - val_loss: 0.0796
Epoch 11/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0503 - val_loss: 0.0761
Epoch 12/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0

In [193]:
y_pred1 = model1.predict(x1)
y_pred0 = model1.predict(x_valid0)

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


In [194]:
mean_absolute_error0 = mse(y_pred0, y_valid0)
mean_absolute_error1 = mse(y_pred1, y1)

In [195]:
mean_absolute_error0

<tf.Tensor: shape=(), dtype=float32, numpy=0.041722845286130905>

In [196]:
mean_absolute_error1

<tf.Tensor: shape=(), dtype=float32, numpy=0.079305000603199>

ADVERSINAL TRAINING

In [99]:
# model1 = tf.keras.Sequential([
#     GRU(32, return_sequences=True, kernel_regularizer=l2(0.001)),  # L2 Regularization
#     Dropout(0.3),  # 30% neurons ko random drop karenge
#     GRU(25, return_sequences=False, kernel_regularizer=l2(0.001)),
#     Dropout(0.3),
#     Dense(1)  # Final output layer
# ])

In [100]:
# model1.compile(loss='mean_squared_error',optimizer='adam')

In [101]:
# mse = MeanSquaredError()
# def create_adversarial_pattern(model, x, y_true, epsilon=0.01, alpha=0.001, num_steps=7):
#     x = tf.convert_to_tensor(x, dtype=tf.float32)
#     y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)

#     # Initialize adversarial example with the original input
#     adversarial_x = tf.identity(x)

#     for _ in range(num_steps):
#         with tf.GradientTape() as tape:
#             tape.watch(adversarial_x)
#             y_pred = model(adversarial_x, training=True)
#             loss = mse(y_true, y_pred)

#         # Compute the gradient
#         gradient = tape.gradient(loss, adversarial_x)
#         signed_grad = tf.sign(gradient)  # Get sign of the gradient

#         # Apply small perturbation step
#         adversarial_x = adversarial_x + alpha * signed_grad

#         # Project the adversarial example to ensure it stays in the epsilon-ball
#         perturbation = tf.clip_by_value(adversarial_x - x, -epsilon, epsilon)
#         adversarial_x = tf.clip_by_value(x + perturbation, 0, 1)  # Ensure valid input range

#     return adversarial_x.numpy()

In [102]:
# x_train_adv = create_adversarial_pattern(model1, x_train0, y_train0)

In [111]:
# x_valid_adv = create_adversarial_pattern(model1, x_valid0, y_valid0)

In [112]:
# x_train_adv.shape

(290, 60, 11)

In [113]:
# x_train_combined = np.vstack([x_train0, x_train_adv])
# y_train_combined = np.hstack([y_train0, y_train0])

In [114]:
# x_valid_combined = np.vstack([x_valid0, x_valid_adv])
# y_valid_combined = np.hstack([y_valid0, y_valid0])

In [115]:
# model1.fit(x_train_combined, y_train_combined, validation_data = (x_valid_combined, y_valid_combined), epochs = 17)

Epoch 1/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0326 - val_loss: 0.0413
Epoch 2/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0339 - val_loss: 0.0401
Epoch 3/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0317 - val_loss: 0.0402
Epoch 4/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0329 - val_loss: 0.0410
Epoch 5/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0296 - val_loss: 0.0410
Epoch 6/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0294 - val_loss: 0.0412
Epoch 7/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0273 - val_loss: 0.0405
Epoch 8/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0274 - val_loss: 0.0425
Epoch 9/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0266 - val_loss: 0.0445
Epoch 10/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0260 - val_loss: 0.0515
Epoch 11/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0249 - val_loss: 0.0645
Epoch 12/17
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0

In [116]:
# y_pred_adv0 = model1.predict(x_valid0)
# y_pred_adv1= model1.predict(x1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [117]:
# mean_absolute_error0 = mse(y_pred_adv0, y_valid0)
# mean_absolute_error1= mse(y_pred_adv1, y1)

In [118]:
# mean_absolute_error0

<tf.Tensor: shape=(), dtype=float32, numpy=0.08647969365119934>

In [119]:
# mean_absolute_error1

<tf.Tensor: shape=(), dtype=float32, numpy=0.1280135214328766>